In [1]:
import pandas as pd
import praw
from dotenv import load_dotenv
import os
import ollama

In [2]:
load_dotenv()

reddit_client_id = os.getenv("REDDIT_CLIENT_ID")
reddit_secret = os.getenv("REDDIT_SECRET")
reddit_user = os.getenv("REDDIT_USER")
reddit_pw = os.getenv("REDDIT_PW")
user_agent = os.getenv("USER_AGENT")

reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_secret,
    user_agent=user_agent,
    username=reddit_user,
    password=reddit_pw
)

In [3]:
subreddit_name = 'news'
subreddit = reddit.subreddit(subreddit_name)

top_posts = subreddit.top(time_filter="day", limit=3)

comment_text_list = []

for post in top_posts:
    post.comments.replace_more(limit=0)
    top_comments = post.comments[:10]

    comments_text = "\n\n".join(
        [f"Comment {i+1} (by u/{c.author}):\n{c.body.strip()}" for i, c in enumerate(top_comments)]
    )

    prompt = f"""Please summarize the following Reddit comments from the post titled: "{post.title}"

    {comments_text}

    Please add additional context, info, or even fact-checking as needed. Use bullet points if helpful.
    """

    comment_text_list.append({
        "post_title": post.title,
        "post_url": post.url,
        "prompt": prompt
    })

comment_text_list[0]

{'post_title': 'Duck Dynasty founding father Phil Robertson dies at 79',
 'post_url': 'https://www.nbcnews.com/news/obituaries/duck-dynasty-founding-father-phil-robertson-dies-79-rcna209035',
 'prompt': 'Please summarize the following Reddit comments from the post titled: "Duck Dynasty founding father Phil Robertson dies at 79"\n\n    Comment 1 (by u/None):\n[deleted]\n\nComment 2 (by u/Nellancher):\nWhat\'s crazy is he went to college and played QB and a very high level\n\nComment 3 (by u/Matman161):\nGoldnecks: rich people who like pretending they\'re just some country boys\n\nComment 4 (by u/Overall-Garbage-254):\nClaims to be Pro-Life, dies.\n\nComment 5 (by u/beefdx):\n“Two guys break into an atheist’s home, he has a little atheist wife and two little atheist daughters. Two guys break into his home, and tie him up in a chair and gag him. And then they take his two daughters in front of him and rape both of ‘em and then shoot ‘em. And they take his wife and decapitate her head off 

In [12]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

for i, prompt in enumerate(comment_text_list, start=0):
    print(f"Title {i + 1}: {prompt['post_title']}")
    print(f"URL {i + 1}: {prompt['post_url']}")
    
    content = comment_text_list[i]['prompt']
    
    messages = [
        {"role": "user", "content": content}
    ]
    
    response = ollama.chat(model=MODEL, messages=messages)
    print(response['message']['content'])
    print("=" * 80)

Title 1: Duck Dynasty founding father Phil Robertson dies at 79
URL 1: https://www.nbcnews.com/news/obituaries/duck-dynasty-founding-father-phil-robertson-dies-79-rcna209035
Here's a summary of the Reddit comments about Phil Robertson:

**Tone and opinions:**

* Most comments express disdain and negativity towards Phil Robertson, with some using strong language to describe him as "garbage" and "a piece of garbage."
* Some commenters accuse Robertson of being a racist (Comment 6) or a hypocrite for claiming to be Pro-Life while dying.
* Others mock his family's lifestyle, comparing them to the "Hillbilly Kardashians" (Comment 7).

**Personal anecdotes:**

* Commenter u/Nellancher mentions that Phil Robertson played college football and attended college.

**Quotes and satire:**

* The comment featuring a satirical quote from Phil Robertson is used to express outrage and disgust towards his views. The quote describes a violent and dehumanizing scenario.
* Commenters use the quote as an op